## **PRODUCT CATEGORY CLASSIFICATION**

### Here, we want to develop an automatic and scalable first prototipe that helps to correctly categorize a new product in the available categories when it arrives.

This first prototipe will help us to identify in what elements we have to go deeper in order to get the best model.


https://towardsdatascience.com/natural-language-processing-in-apache-spark-using-nltk-part-2-2-5550b85f3340

## **Summary of requirements**

1. Train a model that predicts the product category for Software, Digital Software, and
Digital Video Games products using the Amazon Customer Reviews dataset.
2. Evaluate and validate your model.

In [1]:
# I checked warnings, but for the final report I prefer ignore those 
#that really does not affect the results (warnings of libraries, etc)
import warnings
warnings.simplefilter('ignore')

## **Load all the required libraries**

In [2]:
!pip install wordcloud

In [3]:
from pyspark import SparkContext
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
from functools import reduce
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud 
import pandas as pd
import re
import string

## **Step 1: Create spark session and provide master as yarn-client and provide application name.**

In [4]:
# Configuration properties of Apache Spark
#sc.stop()
from pyspark import SparkConf
from pyspark.sql import SparkSession

APP_NAME = 'pyspark_python'
MASTER = 'local[*]'

conf = SparkConf().setAppName(APP_NAME)
conf = conf.setMaster(MASTER)
spark = SparkSession.builder.config(conf = conf).getOrCreate()
sc = spark.sparkContext

## **Step 2: Load amazon_alexa.tsv(TSV stands for Tab Separated Values) data into spark dataframe from HDFS.**

In [51]:

schema = StructType([
    StructField("marketplace",  StringType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("product_id",  StringType(), True),
    StructField("product_parent", IntegerType(), True),
    StructField("product_title", StringType(), True),
    StructField("product_category", StringType(), True),
    StructField("star_rating", IntegerType(), True),
    StructField("helpful_votes", IntegerType(), True),
    StructField("total_votes", IntegerType(), True),
    StructField("vine", StringType(), True),
    StructField("verified_purchase", StringType(), True),
    StructField("review_headline", StringType(), True),
    StructField("review_body", StringType(), True),
    StructField("review_date", StringType(), True)])
'''
df_video_games = spark.read\
    .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
    .option("delimiter",",")\
    .option("inferSchema", "True")\
    .option("header", "True")\
    .load('data/amazon_reviews_us_Digital_Video_Games_v1_00.tsv')
'''

df_video_games = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option("delimiter","\t")\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load('data/amazon_reviews_us_Digital_Video_Games_v1_00.tsv')

df_software = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option("delimiter","\t")\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load('data/amazon_reviews_us_Software_v1_00.tsv')

df_digital_software = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option("delimiter","\t")\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load('data/amazon_reviews_us_Digital_Software_v1_00.tsv')

## **MERGE DATA**

In [52]:
df = df_video_games.union(df_software);
df = df.union(df_digital_software);

In [53]:
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...|2015-08-31 00:00:00|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live

In [54]:
names = ['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date']
df = df.toDF(*names)
df.toPandas().head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,21269168,RSH1OZ87OYK92,B013PURRZW,603406193,Madden NFL 16 - Xbox One Digital Code,Digital_Video_Games,2,2,3,N,N,A slight improvement from last year.,I keep buying madden every year hoping they ge...,2015-08-31
1,US,133437,R1WFOQ3N9BO65I,B00F4CEHNK,341969535,Xbox Live Gift Card,Digital_Video_Games,5,0,0,N,Y,Five Stars,Awesome,2015-08-31
2,US,45765011,R3YOOS71KM5M9,B00DNHLFQA,951665344,Command & Conquer The Ultimate Collection [Ins...,Digital_Video_Games,5,0,0,N,Y,Hail to the great Yuri!,If you are prepping for the end of the world t...,2015-08-31
3,US,113118,R3R14UATT3OUFU,B004RMK5QG,395682204,Playstation Plus Subscription,Digital_Video_Games,5,0,0,N,Y,Five Stars,Perfect,2015-08-31
4,US,22151364,RV2W9SGDNQA2C,B00G9BNLQE,640460561,Saints Row IV - Enter The Dominatrix [Online G...,Digital_Video_Games,5,0,0,N,Y,Five Stars,Awesome!,2015-08-31


## **Step 3: Fetch column: “verified_reviews” because we need only that column for extracting sentiments from customers and for that we need to convert our dataframe into RDD(best suited for processing unstructured data)**

In [9]:
reviews_rdd = df.select("review_body").rdd.flatMap(lambda x: x)

In [10]:
#reviews_rdd.collect()

## **Step 4: Remove header and convert all the data into lowercase for easy processing**

In [11]:
header = reviews_rdd.first()
header

"I keep buying madden every year hoping they get back to football. This years version is a little better than last years -- but that's not saying much.The game looks great. The only thing wrong with the animation, is the way the players are always tripping on each other.<br /><br />The gameplay is still slowed down by the bloated pre-play controls. What used to take two buttons is now a giant PITA to get done before an opponent snaps the ball or the play clock runs out.<br /><br />The turbo button is back, but the player movement is still slow and awkward. If you liked last years version, I'm guessing you'll like this too. I haven't had a chance to play anything other than training and a few online games, so I'm crossing my fingers and hoping the rest is better.<br /><br />The one thing I can recommend is NOT TO BUY THE MADDEN BUNDLE. The game comes as a download. So if you hate it, there's no trading it in at Gamestop."

In [13]:
#data_rmv_col = reviews_rdd.filter(lambda row: row != header)

lowerCase_sentRDD = reviews_rdd.map(lambda x : x.lower())

In [14]:
lowerCase_sentRDD.first()

"i keep buying madden every year hoping they get back to football. this years version is a little better than last years -- but that's not saying much.the game looks great. the only thing wrong with the animation, is the way the players are always tripping on each other.<br /><br />the gameplay is still slowed down by the bloated pre-play controls. what used to take two buttons is now a giant pita to get done before an opponent snaps the ball or the play clock runs out.<br /><br />the turbo button is back, but the player movement is still slow and awkward. if you liked last years version, i'm guessing you'll like this too. i haven't had a chance to play anything other than training and a few online games, so i'm crossing my fingers and hoping the rest is better.<br /><br />the one thing i can recommend is not to buy the madden bundle. the game comes as a download. so if you hate it, there's no trading it in at gamestop."

## **Step 5: Text data can be split into sentences and this process is called sentence tokenization.**

In [15]:
def sent_TokenizeFunct(x):
    return nltk.sent_tokenize(x)
sentenceTokenizeRDD = lowerCase_sentRDD.map(sent_TokenizeFunct)

In [16]:
lowerCase_sentRDD.first()

"i keep buying madden every year hoping they get back to football. this years version is a little better than last years -- but that's not saying much.the game looks great. the only thing wrong with the animation, is the way the players are always tripping on each other.<br /><br />the gameplay is still slowed down by the bloated pre-play controls. what used to take two buttons is now a giant pita to get done before an opponent snaps the ball or the play clock runs out.<br /><br />the turbo button is back, but the player movement is still slow and awkward. if you liked last years version, i'm guessing you'll like this too. i haven't had a chance to play anything other than training and a few online games, so i'm crossing my fingers and hoping the rest is better.<br /><br />the one thing i can recommend is not to buy the madden bundle. the game comes as a download. so if you hate it, there's no trading it in at gamestop."

## **Step 6: Now split each sentence into words, also called word tokenization.**

In [17]:
def word_TokenizeFunct(x):
    splitted = [word for line in x for word in line.split()]
    return splitted
wordTokenizeRDD = sentenceTokenizeRDD.map(word_TokenizeFunct)

In [18]:
wordTokenizeRDD.first()

['i',
 'keep',
 'buying',
 'madden',
 'every',
 'year',
 'hoping',
 'they',
 'get',
 'back',
 'to',
 'football.',
 'this',
 'years',
 'version',
 'is',
 'a',
 'little',
 'better',
 'than',
 'last',
 'years',
 '--',
 'but',
 "that's",
 'not',
 'saying',
 'much.the',
 'game',
 'looks',
 'great.',
 'the',
 'only',
 'thing',
 'wrong',
 'with',
 'the',
 'animation,',
 'is',
 'the',
 'way',
 'the',
 'players',
 'are',
 'always',
 'tripping',
 'on',
 'each',
 'other.<br',
 '/><br',
 '/>the',
 'gameplay',
 'is',
 'still',
 'slowed',
 'down',
 'by',
 'the',
 'bloated',
 'pre-play',
 'controls.',
 'what',
 'used',
 'to',
 'take',
 'two',
 'buttons',
 'is',
 'now',
 'a',
 'giant',
 'pita',
 'to',
 'get',
 'done',
 'before',
 'an',
 'opponent',
 'snaps',
 'the',
 'ball',
 'or',
 'the',
 'play',
 'clock',
 'runs',
 'out.<br',
 '/><br',
 '/>the',
 'turbo',
 'button',
 'is',
 'back,',
 'but',
 'the',
 'player',
 'movement',
 'is',
 'still',
 'slow',
 'and',
 'awkward.',
 'if',
 'you',
 'liked',
 'las

## **Step 7: To move ahead first we will clean our data, here we’re gonna remove stopwords, punctuations and empty spaces.**

In [19]:
def removeStopWordsFunct(x):
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words('english'))
    filteredSentence = [w for w in x if not w in stop_words]
    return filteredSentence
stopwordRDD = wordTokenizeRDD.map(removeStopWordsFunct)

def removePunctuationsFunct(x):
    list_punct=list(string.punctuation)
    filtered = [''.join(c for c in s if c not in list_punct) for s in x] 
    filtered_space = [s for s in filtered if s] #remove empty space 
    return filtered
rmvPunctRDD = stopwordRDD.map(removePunctuationsFunct)

In [20]:
stopwordRDD.first()

['keep',
 'buying',
 'madden',
 'every',
 'year',
 'hoping',
 'get',
 'back',
 'football.',
 'years',
 'version',
 'little',
 'better',
 'last',
 'years',
 '--',
 "that's",
 'saying',
 'much.the',
 'game',
 'looks',
 'great.',
 'thing',
 'wrong',
 'animation,',
 'way',
 'players',
 'always',
 'tripping',
 'other.<br',
 '/><br',
 '/>the',
 'gameplay',
 'still',
 'slowed',
 'bloated',
 'pre-play',
 'controls.',
 'used',
 'take',
 'two',
 'buttons',
 'giant',
 'pita',
 'get',
 'done',
 'opponent',
 'snaps',
 'ball',
 'play',
 'clock',
 'runs',
 'out.<br',
 '/><br',
 '/>the',
 'turbo',
 'button',
 'back,',
 'player',
 'movement',
 'still',
 'slow',
 'awkward.',
 'liked',
 'last',
 'years',
 'version,',
 "i'm",
 'guessing',
 'like',
 'too.',
 'chance',
 'play',
 'anything',
 'training',
 'online',
 'games,',
 "i'm",
 'crossing',
 'fingers',
 'hoping',
 'rest',
 'better.<br',
 '/><br',
 '/>the',
 'one',
 'thing',
 'recommend',
 'buy',
 'madden',
 'bundle.',
 'game',
 'comes',
 'download.',
 

## **Step 8: Lemmatization**
Stemming and Lemmatization are the basic text processing methods for English text. The goal of both of them is to reduce inflectional forms and sometimes derivationally related forms of a word to a common base form. I have skipped Stemming because it is not an efficient method as sometimes it produces words that are not even close to the actual word.


In [21]:
def lemmatizationFunct(x):
    nltk.download('wordnet')
    lemmatizer = WordNetLemmatizer()
    finalLem = [lemmatizer.lemmatize(s) for s in x]
    return finalLem
lem_wordsRDD = rmvPunctRDD.map(lemmatizationFunct)

In [22]:
lem_wordsRDD.first()

['keep',
 'buying',
 'madden',
 'every',
 'year',
 'hoping',
 'get',
 'back',
 'football',
 'year',
 'version',
 'little',
 'better',
 'last',
 'year',
 '',
 'thats',
 'saying',
 'muchthe',
 'game',
 'look',
 'great',
 'thing',
 'wrong',
 'animation',
 'way',
 'player',
 'always',
 'tripping',
 'otherbr',
 'br',
 'the',
 'gameplay',
 'still',
 'slowed',
 'bloated',
 'preplay',
 'control',
 'used',
 'take',
 'two',
 'button',
 'giant',
 'pita',
 'get',
 'done',
 'opponent',
 'snap',
 'ball',
 'play',
 'clock',
 'run',
 'outbr',
 'br',
 'the',
 'turbo',
 'button',
 'back',
 'player',
 'movement',
 'still',
 'slow',
 'awkward',
 'liked',
 'last',
 'year',
 'version',
 'im',
 'guessing',
 'like',
 'too',
 'chance',
 'play',
 'anything',
 'training',
 'online',
 'game',
 'im',
 'crossing',
 'finger',
 'hoping',
 'rest',
 'betterbr',
 'br',
 'the',
 'one',
 'thing',
 'recommend',
 'buy',
 'madden',
 'bundle',
 'game',
 'come',
 'download',
 'hate',
 'it',
 'there',
 'trading',
 'gamestop']

## **Step 9: Our next task is a little tricky, we have to extract key phrases(also called Noun phrases). So first we need to join “lem_wordsRDD” tokens.**

In [23]:
def joinTokensFunct(x):
    joinedTokens_list = []
    x = " ".join(x)
    return x
joinedTokens = lem_wordsRDD.map(joinTokensFunct)

In [25]:
joinedTokens.first()

'keep buying madden every year hoping get back football year version little better last year  thats saying muchthe game look great thing wrong animation way player always tripping otherbr br the gameplay still slowed bloated preplay control used take two button giant pita get done opponent snap ball play clock run outbr br the turbo button back player movement still slow awkward liked last year version im guessing like too chance play anything training online game im crossing finger hoping rest betterbr br the one thing recommend buy madden bundle game come download hate it there trading gamestop'

## **Step 10: Extract key phrases, In the below code I’m doing chunking, chinking and POS tagging using Regular expression and extracting all the Noun phrases.**

* **Chunking:** Also referred to as shallow parsing, is a task that follows Part-Of-Speech Tagging and that adds more structure to the sentence. The result is a grouping of the words in “chunks”.
* **Chinking:** Chinking is a lot like chunking, it is basically a way for you to remove a chunk from a chunk. The chunk that you remove from your chunk is your chink.
* **POS tagging:** A Part-Of-Speech Tagger (POS Tagger) is a piece of software that reads the text in some language and assigns parts of speech to each word (and other tokens), such as noun, verb, adjective, etc.


In [27]:
def extractPhraseFunct(x):
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words('english'))
    def leaves(tree):
        """Finds NP (nounphrase) leaf nodes of a chunk tree."""
        for subtree in tree.subtrees(filter = lambda t: t.label()=='NP'):
            yield subtree.leaves()
    
    def get_terms(tree):
        for leaf in leaves(tree):
            term = [w for w,t in leaf if not w in stop_words]
            yield term
    sentence_re = r'(?:(?:[A-Z])(?:.[A-Z])+.?)|(?:\w+(?:-\w+)*)|(?:\$?\d+(?:.\d+)?%?)|(?:...|)(?:[][.,;"\'?():-_`])'
    grammar = r"""
    NBAR:
        {<NN.*|JJ>*<NN.*>}  # Nouns and Adjectives, terminated with Nouns
        
    NP:
        {<NBAR>}
        {<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...
    """
    chunker = nltk.RegexpParser(grammar)
    tokens = nltk.regexp_tokenize(x,sentence_re)
    postoks = nltk.tag.pos_tag(tokens) #Part of speech tagging 
    tree = chunker.parse(postoks) #chunking
    terms = get_terms(tree)
    temp_phrases = []
    for term in terms:
        if len(term):
            temp_phrases.append(' '.join(term))
    
    finalPhrase = [w for w in temp_phrases if w] #remove empty lists
    return finalPhrase
extractphraseRDD = joinedTokens.map(extractPhraseFunct)

In [28]:
extractphraseRDD.first()

['buying madden',
 'year',
 'football year version',
 'last year thats',
 'muchthe game look great thing wrong animation way player',
 'gameplay',
 'bloated preplay control',
 'button giant pita',
 'opponent snap ball play clock',
 'turbo button',
 'player movement',
 'last year version',
 'chance play anything',
 'online game im',
 'finger',
 'rest betterbr',
 'thing recommend',
 'madden bundle game',
 'download hate',
 'gamestop']

## **Step 11: From the above step we roughly got all the key phrases the customers are talking about. Now categorize these key phrases into positive, Negative or Neutral.**
There are so many ways to get the sentiments, I’m using NLTK VADER you guys can choose any other method as per your interest.

In [31]:
def sentimentWordsFunct(x):
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    analyzer = SentimentIntensityAnalyzer() 
    senti_list_temp = []
    for i in x:
        y = ''.join(i) 
        vs = analyzer.polarity_scores(y)
        senti_list_temp.append((y, vs))
        senti_list_temp = [w for w in senti_list_temp if w]
    sentiment_list  = []
    for j in senti_list_temp:
        first = j[0]
        second = j[1]
    
        for (k,v) in second.items():
            if k == 'compound':
                if v < 0.0:
                    sentiment_list.append((first, "Negative"))
                elif v == 0.0:
                    sentiment_list.append((first, "Neutral"))
                else:
                    sentiment_list.append((first, "Positive"))
     
    return sentiment_list

In [34]:
sentimentRDD = extractphraseRDD.map(sentimentWordsFunct)

In [46]:
extractphraseRDD.first()

['buying madden',
 'year',
 'football year version',
 'last year thats',
 'muchthe game look great thing wrong animation way player',
 'gameplay',
 'bloated preplay control',
 'button giant pita',
 'opponent snap ball play clock',
 'turbo button',
 'player movement',
 'last year version',
 'chance play anything',
 'online game im',
 'finger',
 'rest betterbr',
 'thing recommend',
 'madden bundle game',
 'download hate',
 'gamestop']

## **Step 12: Now let's extract the top 20 keywords from the extracted key phrases.**

In [40]:
freqDistRDD = extractphraseRDD.flatMap(lambda x : nltk.FreqDist(x).most_common()).map(lambda x: x).reduceByKey(lambda x,y : x+y).sortBy(lambda x: x[1], ascending = False)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 22.0 failed 1 times, most recent failure: Lost task 0.0 in stage 22.0 (TID 49, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/erikapat/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/home/erikapat/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/erikapat/anaconda3/lib/python3.7/site-packages/pyspark/rdd.py", line 2499, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/erikapat/anaconda3/lib/python3.7/site-packages/pyspark/rdd.py", line 2499, in pipeline_func
    return func(sp

## **Step 13: Visualizing the output. I’m using the function pandD.plot.barh() to visualize the output. Click this to see all available graphs that can be used instead of a horizontal bar graph(barh).**

In [41]:
df_fDist = freqDistRDD.toDF() #converting RDD to spark dataframe
df_fDist.createOrReplaceTempView("myTable") 
df2 = spark.sql("SELECT _1 AS Keywords, _2 as Frequency from myTable limit 20") #renaming columns 
pandD = df2.toPandas() #converting spark dataframes to pandas dataframes
pandD.plot.barh(x='Keywords', y='Frequency', rot=1, figsize=(10,8))

NameError: name 'freqDistRDD' is not defined